Implementation of the following paper: https://arxiv.org/pdf/0912.3995.pdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from IPython import display
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
y = np.sin(2 * math.pi * x)

In [ ]:
def k_gaussian(x1, x2):
    # calculating gaussian kernel matrix
    # the output has shape (len(x2), len(x1))
    # entry at [i, j] position is given by k(x2[i], x1[j])
    
    # gaussian kernel hyperparameters - adjusts the distance between points and variance
    l = 0.1
    sigma = 1
    
    x1_matrix = np.tile(x1, len(x2)).reshape((len(x2), len(x1)))
    x2_matrix = np.tile(x2, len(x1)).reshape((len(x1), len(x2))).transpose()
    
    k_matrix = np.exp(-(x1_matrix - x2_matrix) ** 2 / (2 * l * l)) * sigma ** 2
    
    return k_matrix

In [ ]:
def gp_posterior(sample_x, sample_y, x):
    # calculating posterior for gaussian processes
    # it is assumed that observations have some additional gaussian noise
    
    sigma_obs = 0.1  
    
    # Separately calculating matrix used to calculate both mean and variance
    K = np.dot(k_gaussian(sample_x, x),
               np.linalg.inv(k_gaussian(sample_x, sample_x) + np.eye(len(sample_x)) * sigma_obs ** 2)
              )
    
    mu = np.dot(K, sample_y)
    sigma = k_gaussian(x, x) - np.dot(K, k_gaussian(x, sample_x))
    
    return mu, sigma

Picking an arbitrary point for a start and setting a loop to do bayesian optimization

In [ ]:
sigma_obs = 0.1

sample_x = np.random.choice(x, size=1, replace=False)
sample_y = np.sin(2 * math.pi * sample_x) + sigma_obs * np.random.randn(1)

print sample_x, sample_y

In [ ]:
delta = 0.1

for t in range(1, 100):
    
    mu, sigma = gp_posterior(sample_x, sample_y, x)
    std_1d = np.sqrt([sigma[i, i] for i in range(len(mu))])
    
    beta_t = 2 * np.log(2 * (t * np.pi) ** 2 / (3 * delta))

    pick_x = np.argmax(mu + np.sqrt(beta_t) * std_1d)
    sample_x = np.append(sample_x, x[pick_x])
    sample_y = np.append(sample_y, np.sin(2 * math.pi * x[pick_x]) + sigma_obs * np.random.randn(1))
    

    plt.plot(x, y, label="true_signal")
    plt.plot(sample_x, sample_y, ".", color="r", label="picked_x")
    plt.plot([sample_x[-1]], [sample_y[-1]], ".", color="b", label="last_x")
    plt.plot(x, mu, color="g", label="posterior")
    plt.fill_between(x, mu - 2 * std_1d, mu + 2 * std_1d, color="g", alpha=0.5)

    plt.title("True and recovered signals")
    plt.legend()
    plt.xlabel("x")
    plt.show()

    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(2)
    